# Project: Italian Restaurants in Toronto

### Business Case:

A manager of an italian restaurant chain (higher middle price category) wants to extend her business  in Toronto. For the first flagstore she is looking for a neighborhood where it seems to be interesting to set up the location. Therefore she wants to know if there is a need for another new italian restaurant. If there is a need she wants to know the best neighborhood or borough to establish the restaurant. She is asking a market research agency to get the relevant information she can base her information on. 

### Key Question: 

Before starting a detailed market research with customers of existing italian restaurants the market research agency wants to identify if there are areas which have a need for a new middle price category restaurants and, if so, which areas should be included in the detailed market survey.

### Analytic Approach: 

The market research agency assumes that first of all it has to identify areas of Toronto where italian restaurants are still existing. The idea behind this approach is that italian restaurants are established in Toronto for a long time. Therefore only in areas, where they are still existing, there is a need for italian restaurant food. In all other areas the italian food has no chance to be established since, if tried before, they did not succeed in surviving. In the next steps the relevant italian restaurants venues have to be clustered by rating and price category. The result has to be represented in a visualization and areas with clusters of a small amount of middle price catagory italian restaurants or a middle to high amount of middle price category restaurants with bad ratings has to be defined. In this areas the market research company will start their field research.

### Data Requirements

To cluster italian restaurants and to localize them in areas we need a representative list of italian restaurants with price categories and ratings and their coordinates. For mapping the coordinates with neighborhoods and boroughs we need a list with geospital data of Toronto.
We assume for this exercise that in the Foursquare application we find a representative view of italian restaurants in Toronto (out of a statistical view we could verify this assumption for example by comparing the amount of italian restaurants in Toronto represented in Foursquare to the total of registered italian restaunrants in Toronto.) For the analytic part we use the venue and location information we get from the Forsquare api and combine it with the neighborhood and borough coordinates we receive from the prepared and downloaded csv file: 'Geospatial_Coordinates.csv'


### Data Collection and Preparation

#### Import required libraries

In [2]:
#import libraries required
import lxml
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import csv
import numpy as np
import requests
import geocoder
from geopy.geocoders import Nominatim
import folium
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import random
from IPython.display import Image 
from IPython.core.display import HTML
import math

#### Define and visualize position of Toronto


First we have to get a geolocation of Toronto to define the radius for the Foursquare request. Looking up Toronto in Wikipedia (https://en.wikipedia.org/wiki/Toronto) reveals that Toronto covers an area of 630 square kilometres. Assuming the complete area is the surface area of a circle the radius around the center is around 14000 m.
First I'll look up the geolocation of Toronto by using the geolocator and than I'll visualize the position in a map with the folium library to decide which radius I have to select for the Foursquare request to cover more or less the whole Toronto area.

In [3]:
# Calculate radius based on circle surface area

radius = math.sqrt(630_000_000/math.pi)
radius1 = round(radius, 0)
print(str(radius1))

"""Get Geolocation Toronto, ON"""
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

14161.0
43.6534817 -79.3839347


Now I visualize the position with the folium library in a Totonto map.

In [4]:

# Look up and visualize position with folium map
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.CircleMarker( # see folium 0.11.0 documentation not folium.features.... if you have installed folium 0.11.0
    [latitude, longitude],
    radius=3,
    color='red',
    popup='Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(toronto_map)
# display map
toronto_map

The geodata does not represent the center of the area of Toronto so I looked up the distance from Toronto city to Vaughan (https://www.distancecalculator.net/from-vaughan-to-toronto) to find the right center of the circle.  I found the following geodata: 43.7001, -79.4163. 
Now I'll have try with this data.

In [5]:
latitude1=43.7001
longitude1=-79.4163


# Look up and visualize position with folium map
toronto_map = folium.Map(location=[latitude1, longitude1], zoom_start=13)

folium.CircleMarker( 
    [latitude1, longitude1],
    radius=3,
    color='red',
    popup='Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(toronto_map)
# display map
toronto_map

These geodata seems to be a better fit to start a search for "Italian Restaurants" with a radius of 14000m in the Foursquare api.

#### Search for Italian Restaurants in Toronto

First I set up the variables I need for the search request in Foursquare. I use the explore venues to get the total count of hits for the category '4bf58dd8d48988d110941735' (see Foursquare documentation categories https://developer.foursquare.com/docs/build-with-foursquare/categories/) = "Italian Restaurants"). Since I expect to get more than 50 hits I need a request form that allows to browse and read all results. With "search" there is no such a pagination function. With "explore" I can use the "Italian Restaurant Category" as well as with "offset" a variable for starting a loop through the pages and adding the results to a list. I select the values for offset as you can see below. I'll use the categoryID for exploring  for the category "Italian Restaurants" in the Foursquare api.

In [6]:
"""Credentials"""
CLIENT_ID = 'XXXXXXXXXXX' # Foursquare ID
CLIENT_SECRET = 'XXXXXXXXX' # Foursquare Secret, left out because of security reasons
VERSION = '20200901'
LIMIT = 50
OFFSET = [0,50,100,150,200] # pagination parameters means start with number 0, 50 etc of the result list
CATID = '4bf58dd8d48988d110941735' # in documentation Id for italian restaurants
#QUERY = 'Italian'
RADIUS = 14000
PRICE = [3]
#print(OFFSET[2])

As described above I repeat the venue search in the "for" loop as long I get all search results for category "Italian Restaurants, radius 14000m around the "center" of Toronto. As the total hit is around 190 I start the last loop with hit number 150. All the hits I append to one list (results1).

In [7]:
"""Search for Italian Restaurants in Foursquare"""

results1 = []

for fig in OFFSET:
    #print(fig)
     
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&offset={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        RADIUS,
        CATID,
        fig,
        LIMIT)
    results = requests.get(url).json()
    results1.append(results)

results1


[{'meta': {'code': 200, 'requestId': '5f5f4b17221cc778cc69c88f'},
  'response': {'suggestedFilters': {'header': 'Tap to show:',
    'filters': [{'name': 'Open now', 'key': 'openNow'}]},
   'headerLocation': 'Toronto',
   'headerFullLocation': 'Toronto',
   'headerLocationGranularity': 'city',
   'query': 'italian',
   'totalResults': 193,
   'suggestedBounds': {'ne': {'lat': 43.77948182600013,
     'lng': -79.21011295664101},
    'sw': {'lat': 43.52748157399987, 'lng': -79.55775644335898}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recommended',
     'items': [{'reasons': {'count': 0,
        'items': [{'summary': 'This spot is popular',
          'type': 'general',
          'reasonName': 'globalInteractionReason'}]},
       'venue': {'id': '4ad776eef964a520e20a21e3',
        'name': 'Mangia and Bevi Resto-Bar',
        'location': {'address': '260 King St E',
         'crossStreet': 'Princess',
         'lat': 43.652249517927686,
         'lng': -79.36635530500347,
 

Now I read the results into one dataframe. First I establish five seperated dataframes for the five seperated  results entries in the results1 list. Therefore I use the the indexes of the single json entries in the list. After that I concatenated the dataframes.

In [8]:
"""Read json in dataframe with explore"""
# assign relevant part of JSON to venues
venues1 = results1[0]['response']['groups'][0]['items']
venues2 = results1[1]['response']['groups'][0]['items']
venues3 = results1[2]['response']['groups'][0]['items']
venues4 = results1[3]['response']['groups'][0]['items']
venues5 = results1[4]['response']['groups'][0]['items']

"""Read json in dataframe with search"""
#venues1 = results1[0]['response']['venues']
#venues2 = results1[1]['response']['venues']
#venues3 = results1[2]['response']['venues']
#venues4 = results1[3]['response']['venues']
#venues5 = results1[4]['response']['venues']
#venues6 = results1[5]['response']['venues']
#venues7 = results1[6]['response']['venues']




# transform venues into a dataframe
list= [venues1, venues2,venues3, venues4, venues5]
df1 = json_normalize(list[0])
df2 = json_normalize(list[1])
df3 = json_normalize(list[2])
df4 = json_normalize(list[3])
df5 = json_normalize(list[4])
#df6 = json_normalize(list[5])
#df7 = json_normalize(list[6])

df = pd.concat([df1, df2, df3, df4, df5],  ignore_index = True)


print(df.columns)
print(df.shape)
df





/home/mathias/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')
(184, 22)


,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad776eef964a520e20a21e3-0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4ad776eef964a520e20a21e3,260 King St E,CA,Toronto,Canada,Princess,1422,"[260 King St E (Princess), Toronto ON M5R 4L5,...","[{'label': 'display', 'lat': 43.65224951792768...",43.652250,-79.366355,NaN,M5R 4L5,ON,Mangia and Bevi Resto-Bar,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ee8f32602d5895bd7dce1b1-1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4ee8f32602d5895bd7dce1b1,101 Portland St,CA,Toronto,Canada,btwn King St W & Adelaide St W,1620,[101 Portland St (btwn King St W & Adelaide St...,"[{'label': 'display', 'lat': 43.64498822340221...",43.644988,-79.400270,NaN,M5V 2N3,ON,Gusto 101,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b9722fef964a52094f934e3-2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4b9722fef964a52094f934e3,875 Queen St. W,CA,Toronto,Canada,NaN,2374,"[875 Queen St. W, Toronto ON M6J 1G5, Canada]","[{'label': 'display', 'lat': 43.64555, 'lng': ...",43.645550,-79.411294,NaN,M6J 1G5,ON,Noce,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4af30f13f964a52030ea21e3-3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4af30f13f964a52030ea21e3,75 Yorkville Ave.,CA,Toronto,Canada,at Bellair St.,2035,"[75 Yorkville Ave. (at Bellair St.), Toronto O...","[{'label': 'display', 'lat': 43.67101871082308...",43.671019,-79.391081,NaN,NaN,ON,Trattoria Nervosa,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b49183ff964a520a46526e3-4,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4b49183ff964a520a46526e3,57 Adelaide St. E,CA,Toronto,Canada,at Church St.,728,"[57 Adelaide St. E (at Church St.), Toronto ON...","[{'label': 'display', 'lat': 43.650927, 'lng':...",43.650927,-79.375602,St. Lawrence,M5C 1K6,ON,Terroni,0,[],NaN
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ade77f6f964a5200c7621e3-5,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4ade77f6f964a5200c7621e3,720 Queen St. W,CA,Toronto,Canada,at Claremont St.,2187,"[720 Queen St. W (at Claremont St.), Toronto O...","[{'label': 'display', 'lat': 43.64617741763388...",43.646177,-79.409150,NaN,M6J 1E8,ON,Terroni,0,[],NaN
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c05ff964a52004f720e3-6,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4ad4c05ff964a52004f720e3,120 Avenue Rd,CA,Toronto,Canada,NaN,2449,"[120 Avenue Rd, Toronto ON M5R 2H4, Canada]","[{'label': 'display', 'lat': 43.67363299036321...",43.673633,-79.396156,NaN,M5R 2H4,ON,Sotto Sotto,0,[],NaN
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-51f70ed7498e22ab07725a43-7,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",51f70ed7498e22ab07725a43,1095 Yonge St.,CA,Toronto,Canada,at Price St.,2985,"[1095 Yonge St. (at Price St.), Toronto ON M4W...","[{'label': 'display', 'lat': 43.67987011343499...",43.679870,-79.390525,NaN,M4W 2L8,ON,Terroni,0,[],NaN
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c060f964a52063f720e3-8,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",4ad4c060f964a52063f720e3,1163 St Clair Avenue West,CA,Toronto,Canada,NaN,5465,"[1163 St Clair Avenue West, Toronto ON M6E 1B2...","[{'label': 'display', 'lat': 43.67801728068154...",43.678017,-79.442725,NaN,M6E 1B2,ON,Marcello's Pizzeria,0,[],NaN
9,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-52f6816f11d24a43115dc834-9,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",52f6816f11d24a43115dc

Now I define the information of interest, filter dataframe and check by venue id if there are duplicates. 



In [9]:
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.formattedAddress', 
                    'venue.location.lat', 'venue.location.lng', 'venue.location.postalCode']
df_filtered = df.loc[:, filtered_columns]
# Search for duplicates
df_duplicates = df_filtered[df_filtered.duplicated(['venue.id'])]
duplicates = df_duplicates['venue.id'].count()
print("Duplicate Rows based on a single column are:" + str(duplicates))
df_filtered

Duplicate Rows based on a single column are:0


,venue.id,venue.name,venue.categories,venue.location.formattedAddress,venue.location.lat,venue.location.lng,venue.location.postalCode
0,4ad776eef964a520e20a21e3,Mangia and Bevi Resto-Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[260 King St E (Princess), Toronto ON M5R 4L5,...",43.652250,-79.366355,M5R 4L5
1,4ee8f32602d5895bd7dce1b1,Gusto 101,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",[101 Portland St (btwn King St W & Adelaide St...,43.644988,-79.400270,M5V 2N3
2,4b9722fef964a52094f934e3,Noce,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[875 Queen St. W, Toronto ON M6J 1G5, Canada]",43.645550,-79.411294,M6J 1G5
3,4af30f13f964a52030ea21e3,Trattoria Nervosa,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[75 Yorkville Ave. (at Bellair St.), Toronto O...",43.671019,-79.391081,NaN
4,4b49183ff964a520a46526e3,Terroni,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[57 Adelaide St. E (at Church St.), Toronto ON...",43.650927,-79.375602,M5C 1K6
5,4ade77f6f964a5200c7621e3,Terroni,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[720 Queen St. W (at Claremont St.), Toronto O...",43.646177,-79.409150,M6J 1E8
6,4ad4c05ff964a52004f720e3,Sotto Sotto,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[120 Avenue Rd, Toronto ON M5R 2H4, Canada]",43.673633,-79.396156,M5R 2H4
7,51f70ed7498e22ab07725a43,Terroni,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[1095 Yonge St. (at Price St.), Toronto ON M4W...",43.679870,-79.390525,M4W 2L8
8,4ad4c060f964a52063f720e3,Marcello's Pizzeria,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[1163 St Clair Avenue West, Toronto ON M6E 1B2...",43.678017,-79.442725,M6E 1B2
9,52f6816f11d24a43115dc834,Scaddabush Italian Kitchen & Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[382 Yonge Street, Unit #7 (Gerrard), Toronto ...",43.658920,-79.382891,M5B 1S8


##### Visualize the position of the restaurants in a Toronto map

Now I visualize the position of restaurants in Toronto by using again the folium library.

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers of restaurants to the map
for lat, lng, name, venueid in zip(df_filtered['venue.location.lat'], df_filtered['venue.location.lng'], 
                                   df_filtered['venue.name'], df_filtered['venue.id']):
    label = '{}, {}'.format(venueid, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

To combine neighborhood data with restaurant data I upload the geolocation data file of Toronto and combine the data with the neighborhood information from wikipedia. (The geolocation file of Toronto I'll upload to my github project account.)

In [11]:



geodata_df = pd.read_csv('Geospatial_Coordinates.csv')
geodata_df.head()
geodata_df.shape
geodata_df.head()

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df_to = pd.read_html(url)
df_to2 = df_to[0]
df_to2.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_to2.head()

#delete cells with  'Not assigned value'
df_to3 = df_to2[df_to2.Borough != "Not assigned"]
df_to3.head()
# reset index
df_to3.reset_index(drop=True,inplace = True)
print(df_to3.shape)
#df_to3.head()
#df_to3.dtypes

# merge dataframes by postal code

df_tomerged = pd.merge(df_to3, geodata_df)
print(df_tomerged.shape)
df_tomerged 

(103, 3)
(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In  the next step I combine the location data with the restaurant data in one map.

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers of restaurants to the map
for lat, lng, name, venueid in zip(df_filtered['venue.location.lat'], df_filtered['venue.location.lng'], 
                                   df_filtered['venue.name'], df_filtered['venue.id']):
    label = '{}, {}'.format(venueid, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
    
for lat1, lng1, borough, neighb in zip(df_tomerged['Latitude'], df_tomerged['Longitude'], 
                                   df_tomerged['Borough'], df_tomerged['Neighborhood']):
    label = '{}, {}'.format(neighb, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat1, lng1],
        radius=15,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FFFFCC',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

##### First preliminary result

On this third visualization we can see which boroughs we can be **excluded**: **Scarborough, York, Etobicoce** (excluding only the neighborhoods in the south close to the coast) and **partially North York**.

### Get venue details

After excluding the first boruoghs the next step is to find information about the price category and rating of the venues ("Italian Restaurants"). Since we are looking for higher middle price category we reduce the hit list by the explore search to price category 3 restaurants (according to Foursquare "https://developer.foursquare.com/docs/api-reference/venues/explore/ price btween USD 20 - 30 in the USA).

In [13]:
#results2 = []

#for fig in OFFSET:
    #print(fig)
     
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}&price={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS,
    CATID,
    LIMIT,
    PRICE[0])
results2 = requests.get(url).json()
results2

{'meta': {'code': 200, 'requestId': '5f5f4c87bf857f310fb1d852'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'query': 'italian',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.77948182600013,
    'lng': -79.21011295664101},
   'sw': {'lat': 43.52748157399987, 'lng': -79.55775644335898}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b49183ff964a520a46526e3',
       'name': 'Terroni',
       'location': {'address': '57 Adelaide St. E',
        'crossStreet': 'at Church St.',
        'lat': 43.650927,
        'lng': -79.375602,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.650927,
          'lng': -79.375602}],
        'distance': 728,
        'postalCode': 'M5

Now I read the json results into one dataframe. Since I receive only 48 hits results2 will cover all data needed and I do not need a loop for reading all results of the request.

In [14]:
"""Read json in dataframe with explore"""
# assign relevant part of JSON to venues
venues6 = results2['response']['groups'][0]['items']
df6 = json_normalize(venues6)
df6

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,e-0-4b49183ff964a520a46526e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b49183ff964a520a46526e3,Terroni,57 Adelaide St. E,at Church St.,43.650927,-79.375602,"[{'label': 'display', 'lat': 43.650927, 'lng':...",728,M5C 1K6,CA,St. Lawrence,Toronto,ON,Canada,"[57 Adelaide St. E (at Church St.), Toronto ON...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
1,e-0-4ada6d36f964a520802221e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ada6d36f964a520802221e3,Pizzeria Libretto,221 Ossington Ave,at Dundas St W,43.648979,-79.420604,"[{'label': 'display', 'lat': 43.64897862710277...",2995,M6J 2Z8,CA,NaN,Toronto,ON,Canada,"[221 Ossington Ave (at Dundas St W), Toronto O...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
2,e-0-51b0a544454ac55245b70ef9-2,0,"[{'summary': 'This spot is popular', 'type': '...",51b0a544454ac55245b70ef9,Cibo Wine Bar King Street,522 King Street West,NaN,43.645073,-79.397360,"[{'label': 'display', 'lat': 43.64507284301734...",1430,M5V 1K4,CA,NaN,Toronto,ON,Canada,"[522 King Street West, Toronto ON M5V 1K4, Can...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
3,e-0-4a8355bff964a520d3fa1fe3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4a8355bff964a520d3fa1fe3,Mercatto,101 College St,NaN,43.660391,-79.387664,"[{'label': 'display', 'lat': 43.66039091189854...",825,M5G,CA,NaN,Toronto,ON,Canada,"[101 College St, Toronto ON M5G, Canada]","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
4,e-0-51f70ed7498e22ab07725a43-4,0,"[{'summary': 'This spot is popular', 'type': '...",51f70ed7498e22ab07725a43,Terroni,1095 Yonge St.,at Price St.,43.679870,-79.390525,"[{'label': 'display', 'lat': 43.67987011343499...",2985,M4W 2L8,CA,NaN,Toronto,ON,Canada,"[1095 Yonge St. (at Price St.), Toronto ON M4W...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
5,e-0-4af30f13f964a52030ea21e3-5,0,"[{'summary': 'This spot is popular', 'type': '...",4af30f13f964a52030ea21e3,Trattoria Nervosa,75 Yorkville Ave.,at Bellair St.,43.671019,-79.391081,"[{'label': 'display', 'lat': 43.67101871082308...",2035,NaN,CA,NaN,Toronto,ON,Canada,"[75 Yorkville Ave. (at Bellair St.), Toronto O...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
6,e-0-4cc3a79bbde8f04d0ddba64b-6,0,"[{'summary': 'This spot is popular', 'type': '...",4cc3a79bbde8f04d0ddba64b,Woodlot Restaurant & Bakery,293 Palmerston Ave.,at College St.,43.655765,-79.409929,"[{'label': 'display', 'lat': 43.655765, 'lng':...",2108,M6J 2J3,CA,NaN,Toronto,ON,Canada,"[293 Palmerston Ave. (at College St.), Toronto...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
7,e-0-4af5c1f0f964a5206efc21e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4af5c1f0f964a5206efc21e3,Buca,604 King St. W,at Portland St.,43.644789,-79.400394,"[{'label': 'display', 'lat': 43.64478896478924...",1641,M5V 1M6,CA,NaN,Toronto,ON,Canada,"[604 King St. W (at Portland St.), Toronto ON ...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
8,e-0-56aabee1498ebfd21c627b88-8,0,"[{'summary': 'This spot is popular', 'type': '...",56aabee1498ebfd21c627b88,Ufficio,1214 Dundas St W,NaN,43.649439,-79.423014,"[{'label': 'display', 'lat': 43.64943939088825...",3179,M6J 1X5,CA,"Dufferin Grove, Toronto, ON",Toronto,ON,Canada,"[1214 Dundas St W, Toronto ON M6J 1X5, Canada]","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
9,e-0-4c2bd80e57a9c9b6b796f667-9,0,"[{'summary': 'This spot is popular', 'type': '...",4c2bd80e57a9c9b6b796f667,Quanto Basta,1112 Yon

To reduce the data to the data we need I filter the dataframe as before.

In [15]:
filtered_columns2 = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.formattedAddress', 
                    'venue.location.lat', 'venue.location.lng', 'venue.location.postalCode']
df_filtered2 = df6.loc[:, filtered_columns2]
# Search for duplicates
df_duplicates2 = df_filtered2[df_filtered2.duplicated(['venue.id'])]
duplicates = df_duplicates['venue.id'].count()
print("Duplicate Rows based on a single column are:" + str(duplicates))
df_filtered2

Duplicate Rows based on a single column are:0


,venue.id,venue.name,venue.categories,venue.location.formattedAddress,venue.location.lat,venue.location.lng,venue.location.postalCode
0,4b49183ff964a520a46526e3,Terroni,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[57 Adelaide St. E (at Church St.), Toronto ON...",43.650927,-79.375602,M5C 1K6
1,4ada6d36f964a520802221e3,Pizzeria Libretto,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[221 Ossington Ave (at Dundas St W), Toronto O...",43.648979,-79.420604,M6J 2Z8
2,51b0a544454ac55245b70ef9,Cibo Wine Bar King Street,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[522 King Street West, Toronto ON M5V 1K4, Can...",43.645073,-79.397360,M5V 1K4
3,4a8355bff964a520d3fa1fe3,Mercatto,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[101 College St, Toronto ON M5G, Canada]",43.660391,-79.387664,M5G
4,51f70ed7498e22ab07725a43,Terroni,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[1095 Yonge St. (at Price St.), Toronto ON M4W...",43.679870,-79.390525,M4W 2L8
5,4af30f13f964a52030ea21e3,Trattoria Nervosa,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[75 Yorkville Ave. (at Bellair St.), Toronto O...",43.671019,-79.391081,NaN
6,4cc3a79bbde8f04d0ddba64b,Woodlot Restaurant & Bakery,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[293 Palmerston Ave. (at College St.), Toronto...",43.655765,-79.409929,M6J 2J3
7,4af5c1f0f964a5206efc21e3,Buca,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[604 King St. W (at Portland St.), Toronto ON ...",43.644789,-79.400394,M5V 1M6
8,56aabee1498ebfd21c627b88,Ufficio,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...","[1214 Dundas St W, Toronto ON M6J 1X5, Canada]",43.649439,-79.423014,M6J 1X5
9,4c2bd80e57a9c9b6b796f667,Quanto Basta,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",[1112 Yonge St (btwn Macpherson Ave & Roxborou...,43.678779,-79.390472,M4W 2L6


In the next step we have to identify the rating for all of the venues (Italian restaurants; price category = 3).

### Get ratings for each venue

To get the rating for each venue I have to loop through all venue id's with explore and get the details information for each venue. Unfortunately the rate limit for the sandbox account for this premium endpoint is 50 requests per day. I can only try one request loop per day (48 hits). Hope it will work today;-).

In [16]:
results3 = []

for vid in df_filtered2['venue.id']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(vid, CLIENT_ID, CLIENT_SECRET, VERSION)
    results4 = requests.get(url).json()
    results3.append(results4)

results3

[{'meta': {'code': 200, 'requestId': '5f5f4d9eb5de2b6f482dd9af'},
  'response': {'venue': {'id': '4b49183ff964a520a46526e3',
    'name': 'Terroni',
    'contact': {'phone': '4162033093',
     'formattedPhone': '(416) 203-3093',
     'twitter': 'terronito'},
    'location': {'address': '57 Adelaide St. E',
     'crossStreet': 'at Church St.',
     'lat': 43.650927,
     'lng': -79.375602,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.650927,
       'lng': -79.375602}],
     'postalCode': 'M5C 1K6',
     'cc': 'CA',
     'neighborhood': 'St. Lawrence',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['57 Adelaide St. E (at Church St.)',
      'Toronto ON M5C 1K6',
      'Canada']},
    'canonicalUrl': 'https://foursquare.com/v/terroni/4b49183ff964a520a46526e3',
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
    

After getting the final result3 list I set up a dataframe. Here I tried a lot of things to automatize the work below. But I did not find a way to avoid so much manual work. Maybe one of my classmates could give me a hint. Flattening the json does not work, because the items are to deep nested in the json file. I have to find a way to loop through the result list, normalize each json list entry and concatenate all that in one dataframe. 

In [21]:
# rows list initialization 
#rows = [] 
  
# appending rows 
#for data in results3: 
    #data_row = data['response']['venue']
    #for row in data_row: 
        #rows.append(row) 
#f001 = json_normalize(rows)  
# using data frame 
#df = pd.DataFrame(rows) 



venue7 = results3[0]['response']['venue']
venue8 = results3[1]['response']['venue']
venue9 = results3[2]['response']['venue']
venue10 = results3[3]['response']['venue']
venue11 = results3[4]['response']['venue']
venue12 = results3[5]['response']['venue']
venue13 = results3[6]['response']['venue']
venue14 = results3[7]['response']['venue']
venue15 = results3[8]['response']['venue']
venue16 = results3[9]['response']['venue']
venue17 = results3[10]['response']['venue']
venue18 = results3[11]['response']['venue']
venue19 = results3[12]['response']['venue']
venue20 = results3[13]['response']['venue']
venue21 = results3[14]['response']['venue']
venue22 = results3[15]['response']['venue']
venue23 = results3[16]['response']['venue']
venue24 = results3[17]['response']['venue']
venue25 = results3[18]['response']['venue']
venue26 = results3[19]['response']['venue']
venue27 = results3[20]['response']['venue']
venue28 = results3[21]['response']['venue']
venue29 = results3[22]['response']['venue']
venue30 = results3[23]['response']['venue']
venue31 = results3[24]['response']['venue']
venue32 = results3[25]['response']['venue']
venue33 = results3[26]['response']['venue']
venue34 = results3[27]['response']['venue']
venue35 = results3[28]['response']['venue']
venue36 = results3[29]['response']['venue']
venue37 = results3[30]['response']['venue']
venue38 = results3[31]['response']['venue']
venue39 = results3[32]['response']['venue']
venue40 = results3[33]['response']['venue']
venue41 = results3[34]['response']['venue']
venue42 = results3[35]['response']['venue']
venue43 = results3[36]['response']['venue']
venue44 = results3[37]['response']['venue']
venue45 = results3[38]['response']['venue']
venue46 = results3[39]['response']['venue']
venue47 = results3[40]['response']['venue']
venue48 = results3[41]['response']['venue']
venue49 = results3[42]['response']['venue']
venue50 = results3[43]['response']['venue']
venue51 = results3[44]['response']['venue']
venue52 = results3[45]['response']['venue']
venue53 = results3[46]['response']['venue']
venue54 = results3[47]['response']['venue']

                  

df7 = json_normalize(venue7)
df8 = json_normalize(venue8)
df9 = json_normalize(venue9)
df10 = json_normalize(venue10)
df11 = json_normalize(venue11)
df12 = json_normalize(venue12)
df13 = json_normalize(venue13)
df14 = json_normalize(venue14)
df15 = json_normalize(venue15)
df16 = json_normalize(venue16)
df17 = json_normalize(venue17)
df18 = json_normalize(venue18)
df19 = json_normalize(venue19)
df20 = json_normalize(venue20)
df21 = json_normalize(venue21)
df22 = json_normalize(venue22)
df23 = json_normalize(venue23)
df24 = json_normalize(venue24)
df25 = json_normalize(venue25)
df26 = json_normalize(venue26)
df27 = json_normalize(venue27)
df28 = json_normalize(venue28)
df29 = json_normalize(venue29)
df30 = json_normalize(venue30)
df31 = json_normalize(venue31)                   
df32 = json_normalize(venue32)
df33 = json_normalize(venue33)
df34 = json_normalize(venue34)
df35 = json_normalize(venue35)
df36 = json_normalize(venue36)
df37 = json_normalize(venue37)
df38 = json_normalize(venue38)
df39 = json_normalize(venue39)
df40 = json_normalize(venue40)
df41 = json_normalize(venue41)
df42 = json_normalize(venue42)
df43 = json_normalize(venue43)
df44 = json_normalize(venue44)
df45 = json_normalize(venue45)
df46 = json_normalize(venue46)
df47 = json_normalize(venue47)
df48 = json_normalize(venue48)
df49 = json_normalize(venue49)
df50 = json_normalize(venue50)
df51 = json_normalize(venue51)
df52 = json_normalize(venue52)
df53 = json_normalize(venue53)
df54 = json_normalize(venue54)

                   
list1 = [df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22,
        df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36, df37, df38, df39,
        df40, df41, df42, df43, df44, df45, df46, df47, df48, df49, df50, df51, df52, df53, df54]


df_rating = pd.concat(list1,  ignore_index = True)                             
                   
df_rating                
                   

/home/mathias/.local/lib/python3.6/site-packages/ipykernel_launcher.py:121: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,allowMenuUrlEdit,attributes.groups,beenHere.count,beenHere.lastCheckinExpiredAt,beenHere.marked,beenHere.unconfirmedCount,bestPhoto.createdAt,bestPhoto.height,bestPhoto.id,bestPhoto.prefix,bestPhoto.source.name,bestPhoto.source.url,bestPhoto.suffix,bestPhoto.visibility,bestPhoto.width,canonicalUrl,categories,colors.algoVersion,colors.highlightColor.photoId,colors.highlightColor.value,colors.highlightTextColor.photoId,colors.highlightTextColor.value,contact.facebook,contact.facebookName,contact.facebookUsername,contact.formattedPhone,contact.instagram,contact.phone,contact.twitter,createdAt,defaultHours.dayData,defaultHours.isLocalHoliday,defaultHours.isOpen,defaultHours.richStatus.entities,defaultHours.richStatus.text,defaultHours.status,defaultHours.timeframes,description,dislike,hasMenu,hereNow.count,hereNow.groups,hereNow.summary,hierarchy,hours.dayData,hours.isLocalHoliday,hours.isOpen,hours.richStatus.entities,hours.richStatus.text,hours.status,hours.timeframes,id,inbox.count,inbox.items,likes.count,likes.groups,likes.summary,listed.count,listed.groups,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,menu.anchor,menu.externalUrl,menu.label,menu.mobileUrl,menu.type,menu.url,name,ok,page.pageInfo.banner,page.pageInfo.description,page.pageInfo.links.count,page.pageInfo.links.items,page.user.bio,page.user.firstName,page.user.id,page.user.lists.groups,page.user.photo.prefix,page.user.photo.suffix,page.user.tips.count,page.user.type,page.user.venue.id,pageUpdates.count,pageUpdates.items,parent.categories,parent.closed,parent.id,parent.location.address,parent.location.cc,parent.location.city,parent.location.country,parent.location.crossStreet,parent.location.formattedAddress,parent.location.labeledLatLngs,parent.location.lat,parent.location.lng,parent.location.postalCode,parent.location.state,parent.name,photos.count,photos.groups,popular.isLocalHoliday,popular.isOpen,popular.timeframes,price.currency,price.message,price.tier,rating,ratingColor,ratingSignals,reasons.count,reasons.items,seasonalHours,shortUrl,specials.count,specials.items,stats.tipCount,storeId,timeZone,tips.count,tips.groups,url,venuePage.id,verified
0,True,"[{'type': 'price', 'name': 'Price', 'summary':...",0,0,False,0,1344989373,612,502ae8bde4b070d90836d029,https://fastly.4sqi.net/img/general/,Instagram,http://instagram.com,/6z112q8kGTbzzaiLECB77CnYHZGG2WKecknQGeS1PJc.jpg,public,612,https://foursquare.com/v/terroni/4b49183ff964a...,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",3.0,502ae8bde4b070d90836d029,-15726560.0,502ae8bde4b070d90836d029,-1.0,NaN,NaN,NaN,(416) 203-3093,NaN,4162033093,terronito,1263081535,[],False,False,[],Closed until 11:00 AM,Closed until 11:00 AM,"[{'days': 'Mon–Thu', 'includesToday': True, 'o...",NaN,False,True,0,[],Nobody here,NaN,[],False,False,[],Closed until 11:00 AM,Closed until 11:00 AM,"[{'days': 'Mon–Thu', 'includesToday': True, 'o...",4b49183ff964a520a46526e3,0,[],279,"[{'type': 'others', 'count': 279, 'items': []}]",279 Likes,222,"[{'type': 'others', 'name': 'Lists from other ...",57 Adelaide St. E,CA,Toronto,Canada,at Church St.,"[57 Adelaide St. E (at Church St.), Toronto ON...","[{'label': 'display', 'lat': 43.650927, 'lng':...",43.650927,-79.375602,St. Lawrence,M5C 1K6,ON,View Menu,NaN,Menu,https://foursquare.com/v/4b49183ff964a520a4652...,Menu,https://foursquare.com/v/terroni/4b49183ff964a...,Terroni,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355,"[{'type': 'venue', 'name': 'Venue photos', 'co...",False,False,"[{'days': 'Today', 'includesToday': True, 'ope...",$,Expensive,3,8.4,73CF42,392.0,1,"[{'summary': 'Lots of people like this place',...",[],http://4sq.com/biJ0I2,0,[],95,NaN,America/Toronto,95,"[{'type': 'others', 'name': 'All tips', 'count...",http://www.terroni.com,Na

Again I filter the dataframe for id, name geolocation, postal code and rating.

In [22]:
filtered_columns3 = ['id', 'name', 'likes.count', 'rating', 'ratingSignals', 
                    'location.lat', 'location.lng', 'location.postalCode']
df_filtered3 = df_rating.loc[:, filtered_columns3]
# Search for duplicates

df_filtered3

,id,name,likes.count,rating,ratingSignals,location.lat,location.lng,location.postalCode
0,4b49183ff964a520a46526e3,Terroni,279,8.4,392.0,43.650927,-79.375602,M5C 1K6
1,4ada6d36f964a520802221e3,Pizzeria Libretto,339,9.2,475.0,43.648979,-79.420604,M6J 2Z8
2,51b0a544454ac55245b70ef9,Cibo Wine Bar King Street,163,8.3,225.0,43.645073,-79.397360,M5V 1K4
3,4a8355bff964a520d3fa1fe3,Mercatto,56,8.1,81.0,43.660391,-79.387664,M5G
4,51f70ed7498e22ab07725a43,Terroni,168,8.7,228.0,43.679870,-79.390525,M4W 2L8
5,4af30f13f964a52030ea21e3,Trattoria Nervosa,204,8.3,296.0,43.671019,-79.391081,NaN
6,4cc3a79bbde8f04d0ddba64b,Woodlot Restaurant & Bakery,83,8.3,139.0,43.655765,-79.409929,M6J 2J3
7,4af5c1f0f964a5206efc21e3,Buca,139,8.2,212.0,43.644789,-79.400394,M5V 1M6
8,56aabee1498ebfd21c627b88,Ufficio,20,8.3,28.0,43.649439,-79.423014,M6J 1X5
9,4c2bd80e57a9c9b6b796f667,Quanto Basta,9,8.0,13.0,43.678779,-79.390472,M4W 2L6


### Cluster by rating


Now I will have a look if a clustering only by rating will divide the restaurants into two groups. One with a low rating and one with a better rating. First I drop the NaN values than I reduce the dataframe only to id and rating. After that I execute the kmeans calculation on two clusters and add the labels to the dataframe. After that I add the labels to the existing dataframe df_filtered3.

In [28]:
df_filtered4 = df_filtered3.dropna()
#df_filtered4
df_clrating = df_filtered4[['rating']]

#df_clrating

kclusters = 2
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clrating)
#k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
#k_means.fit(cluster_dataset)
labels = kmeans.labels_

print(labels)

df_clrating['labels'] = labels
df_clrating







[1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]


/home/mathias/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,rating,labels
0,8.4,1
1,9.2,1
2,8.3,1
3,8.1,1
4,8.7,1
6,8.3,1
7,8.2,1
8,8.3,1
9,8.0,1
11,7.5,0


#### Examine clusters

I check if the cluster make a difference between rating values. First cluster with lable 0. After that cluster with label value 1.

In [31]:
# check if cluster make a difference in rating

df_clrating.loc[df_clrating['labels'] == 0]



,rating,labels
11,7.5,0
15,7.6,0
17,7.5,0
20,7.2,0
21,7.2,0
22,7.2,0
25,7.2,0
26,7.2,0
27,7.1,0
28,7.3,0


In [32]:
df_clrating.loc[df_clrating['labels'] == 1]

,rating,labels
0,8.4,1
1,9.2,1
2,8.3,1
3,8.1,1
4,8.7,1
6,8.3,1
7,8.2,1
8,8.3,1
9,8.0,1
12,7.7,1


We see that the clustering devides in two groups (as assumed). 0 labels for ratings <= 7.6 and 1 for ratings >7.6. This is a good differentiator taking into a account that on a scale of 10  (e.g. Net Promoter Score) ratings around 8 are ratings in the recommendation zone. Normally the ratings have to be weighted also by the rating signals but for this exercise we do not need to go in such depth.

#### Adding labels to df_filtered4

Since the clusters make a significant difference and divide in low rated and high rated restaurants we proceed and add the lablels to df_filtered4 for preparing the final visualization.

In [33]:
# add clustering labels
df_filtered4.insert(8,'Cluster Labels', kmeans.labels_)
df_filtered4

,id,name,likes.count,rating,ratingSignals,location.lat,location.lng,location.postalCode,Cluster Labels
0,4b49183ff964a520a46526e3,Terroni,279,8.4,392.0,43.650927,-79.375602,M5C 1K6,1
1,4ada6d36f964a520802221e3,Pizzeria Libretto,339,9.2,475.0,43.648979,-79.420604,M6J 2Z8,1
2,51b0a544454ac55245b70ef9,Cibo Wine Bar King Street,163,8.3,225.0,43.645073,-79.397360,M5V 1K4,1
3,4a8355bff964a520d3fa1fe3,Mercatto,56,8.1,81.0,43.660391,-79.387664,M5G,1
4,51f70ed7498e22ab07725a43,Terroni,168,8.7,228.0,43.679870,-79.390525,M4W 2L8,1
6,4cc3a79bbde8f04d0ddba64b,Woodlot Restaurant & Bakery,83,8.3,139.0,43.655765,-79.409929,M6J 2J3,1
7,4af5c1f0f964a5206efc21e3,Buca,139,8.2,212.0,43.644789,-79.400394,M5V 1M6,1
8,56aabee1498ebfd21c627b88,Ufficio,20,8.3,28.0,43.649439,-79.423014,M6J 1X5,1
9,4c2bd80e57a9c9b6b796f667,Quanto Basta,9,8.0,13.0,43.678779,-79.390472,M4W 2L6,1
11,4d2b615e342d6dcb2b8115cb,Earls Kitchen & Bar,260,7.5,440.0,43.647946,-79.383706,M5H 2B6,0


With dropping NaN values in the rating field from the list of high middle class Italian restauratants in Toronto we have lost 7 entries around 15 % of the total. This should be mentioned concerning the significance of the research.

### Visualize all Italian restaurants in Toronto and price catagory 3 restaurants with cluster label rating.

After adding the labels to df_filtered4 I'll visualize the restaurants with their rating labels with all restaurants and the Toronto boroughs. 

green = middle priced restaurants with rating lable in lable of the map 

red-circled yellow = neighborhoods, boroughs 

blue = all italian estaurants

In [40]:
# create map of Toronto using latitude and longitude values
map_toronto3 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers of restaurants to the map

for lat, lng, name, venueid in zip(df_filtered['venue.location.lat'], df_filtered['venue.location.lng'], 
                                   df_filtered['venue.name'], df_filtered['venue.id']):
    label = '{}, {}'.format(venueid, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto3)
    
for lat1, lng1, borough, neighb in zip(df_tomerged['Latitude'], df_tomerged['Longitude'], 
                                   df_tomerged['Borough'], df_tomerged['Neighborhood']):
    label = '{}, {}'.format(neighb, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat1, lng1],
        radius=15,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FFFFCC',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto3)  
    
for lat2, lng2, label, name in zip(df_filtered4['location.lat'], df_filtered4['location.lng'], 
                                   df_filtered4['Cluster Labels'], df_filtered4['name']):
    label = '{}, {}'.format(label, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat2, lng2],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#04B404',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto3) 
    
map_toronto3

Now we can see that in the following boroughs there a small number of price category 3 italian restaurants or most of them with low ratings category 0. 

**North York**: low rating category 3 restaurants: esp. Boroughs: **Bedford Park, Lawrence Manor East**
**Central Toronto** low rating category 3 restaurants: esp. Borough **Davisville** 
**Central Toronto** competition with one existing high rated restauran: Borough: **Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park**

Downtown Toronto and West Toronto are very well equipped with every type of Italian restaurant. So there seemed to be not a real opportunity for starting a new  one. 

So we recommend the market research company to start there field research in the above mentioned  boroughs.

## Alternative Route 

If the clustering did not give enough information about the rating groups we would divide the total df_filtered3 in two groups. One group with a average rating lower than 8 and the other with a rating higher than 7.5. After that we visualize these two groups with the boroughs in the toronto_map to see where there could be an opportunity for the field research.

### Sorting Dataframe by rating

In [44]:
df_lowrat = df_filtered3.loc[(df_filtered3['rating'] <= 7.5)]
df_highrat = df_filtered3.loc[(df_filtered3['rating'] > 7.5)]
df_lowrat


,id,name,likes.count,rating,ratingSignals,location.lat,location.lng,location.postalCode
11,4d2b615e342d6dcb2b8115cb,Earls Kitchen & Bar,260,7.5,440.0,43.647946,-79.383706,M5H 2B6
17,4ad4c060f964a52050f720e3,Kalendar Restaurant & Bistro,64,7.5,97.0,43.655672,-79.411561,M6G 1B1
20,4c251062db519521621d2c3a,La Bettola Di Terroni,69,7.2,115.0,43.651993,-79.378056,M5C 2B4
21,4adc5c6af964a520da2b21e3,Mercatto,46,7.2,83.0,43.650243,-79.380820,M5H 2S8
22,4afcc52df964a520b82522e3,Donatello Restaurant,19,7.2,42.0,43.657489,-79.383605,M5G 1H1
25,4ad4c05cf964a52004f620e3,Grano,10,7.2,24.0,43.701712,-79.397318,M4P 2A2
26,4bd4b2546798ef3bb235628d,Florentia Ristorante,7,7.2,10.0,43.703594,-79.387985,M4S 2M5
27,4b108405f964a5200d7223e3,Local Kitchen & Wine Bar,16,7.1,29.0,43.638951,-79.445224,M6R 1B3
28,4e39847881dc123313658c5d,La Veranda Osteria,13,7.3,25.0,43.647638,-79.511442,M8X 2E5
29,50c2488ce4b0fe852ca6ae17,Grappa,5,7.3,8.0,43.627113,-79.499500,NaN


### Visualize results with boroughs

blue-circle green = middle priced, lower rating

red-circled green = middle priced, good rating

green-circled green = all italian restaurants

black-circled yellow = neighborhoods, boroughs

In [50]:
# create map of Toronto using latitude and longitude values
map_toronto4 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers of restaurants to the map

for lat4, lng4, name4, rating4 in zip(df_lowrat['location.lat'], df_lowrat['location.lng'], 
                                   df_lowrat['name'], df_lowrat['rating']):
    label = '{}, {}'.format(rating4, name4)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat4, lng4],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto4)
    
for lat5, lng5, name5, rating5 in zip(df_highrat['location.lat'], df_highrat['location.lng'], 
                                   df_highrat['name'], df_highrat['rating']):
    label = '{}, {}'.format(rating5, name5)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat5, lng5],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FFFFCC',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto4)  
    
for lat2, lng2, rating, name in zip(df_filtered3['location.lat'], df_filtered3['location.lng'], 
                                   df_filtered3['rating'], df_filtered3['name']):
    label = '{}, {}'.format(rating, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat2, lng2],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#04B404',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto4) 

for lat1, lng1, borough, neighb in zip(df_tomerged['Latitude'], df_tomerged['Longitude'], 
                                   df_tomerged['Borough'], df_tomerged['Neighborhood']):
    label = '{}, {}'.format(neighb, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat1, lng1],
        radius=15,
        popup=label,
        color='black',
        fill=True,
        fill_color='#FFFFCC',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto4)

    
map_toronto4

The results of this alternative route confirm the findings we described under the clustering route.

# Recommandation

Start the field research in the following areas. 

**North York**: low rating category 3 restaurants: esp. Boroughs: **Bedford Park, Lawrence Manor East**

**Central Toronto** low rating category 3 restaurants: esp. Borough **Davisville** 

**Central Toronto** competition with one existing high rated restauran: Borough: **Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park**

In these areas there is a need for italian food, since italian restaurants are well established. In addition these areas have a lack of good rated middle priced (category 3) restaurants (**North York**: Boroughs: **Bedford Park, Lawrence Manor East** and **Central Toronto**: Borough **Davisville**) or there is only one competitor which indicates that the customers are willing to pay the price(**Central Toronto**: **Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park** ).